In [117]:
from pymongo import MongoClient
import pprint,sys,re


client = MongoClient()
db = client.lobbyradar
entity_cursor = db.entities.find()


In [118]:
# remove unicode prefix, taken from:
# http://stackoverflow.com/questions/16888409/suppress-unicode-prefix-on-strings-when-using-pprint
class writer :
    def write(self, text):
        text=re.sub(r'u\'([^\']*)\'', r'\1',text)
        sys.stdout.write(text)

wrt=writer()
p = pprint.PrettyPrinter(stream=wrt)

In [119]:
print "Entity Types: " 
p.pprint(db.entities.distinct("type"))

Entity Types: 
[entity, person]


In [120]:
def list_tags():
    p.pprint(db.entities.distinct("tags"))
print "All tags:"
list_tags()

All tags:
[None,
 Architektur,
 Auto / Verkehr,
 Bank,
 Banken / Versicherungen,
 Bau,
 Beton,
 Bundestagsfraktion,
 Bundeswehr,
 Chemie,
 Energie / Wasser / Versorgung,
 Finanzen,
 Gesundheit,
 Gewerkschaft,
 Handel,
 Holz,
 IT,
 Immobilien,
 Industrie,
 Lebensmittel,
 Medien,
 Medien / IT,
 Medizin,
 Metall,
 M\xf6bel,
 Public Affairs und Lobbyagentur,
 Religion,
 R\xfcstung,
 Sport,
 Stahl,
 Umwelt,
 Verlag,
 Versandhandel,
 Verwaltung / Politik,
 Werbung / PR,
 Zement,
 advokatisch,
 akademisch,
 alkohol,
 anwaltskanzlei,
 architektur,
 ard,
 aufsichtsrat,
 auto,
 baden-wurttemberg,
 bank,
 banken,
 bau,
 bayern,
 bergbau,
 berlin,
 bertelsmann,
 beteiligungsgesellschaft,
 bildung,
 brandenburg,
 bremen,
 bundeskanzlerin,
 bundesregierung,
 bundestag,
 bundestagsfraktion,
 bundeswehr,
 cdu,
 chemie,
 committee,
 consultating,
 consulting,
 csu,
 dax,
 eisenbahn,
 elektronik,
 eletronik,
 enenrgie,
 energie,
 executive,
 experte,
 fdp,
 fernsehen,
 film,
 finanzdienstleistungen,
 fi

In [155]:
print db.entities.find({"tags" :{"$in": [ "lobbyismus", "lobbyist", "lobbyorganisation"]}}).count()

17001


In [156]:
print db.entities.find({"tags" :{"$in": [ "parteispenden", "parteispenden13", "parteispenden14"]}}).count()

3536


In [121]:
def display(query, projection):
    data = db.entities.find(query, projection)
    data = data.sort("name") 
    for i in data:
        p.pprint(i)

In [122]:
def entities_find(key, value):
    query = {key: value}
    projection = {"_id":0, "name":1}
    display(query, projection)
   
    

In [123]:
entities_find("tags", "IT")

{name: ArGe PERSER Arbeitsgemeinschaft der Personalabrechnungs-Software-Ersteller}
{name: Arbeitskreis Amateurfunk und Telekommunikation in der Schule e.V. (AATiS)}
{name: BSA Business Software Alliance}
{name: Bundesverband Deutsche Startups e.V.}
{name: Bundesverband IT-Mittelstand e.V. (BITMi)}
{name: Bundesverband Informationswirtschaft, Telekommunikation und neue Medien (BITKOM) e.V.}
{name: Bundesverband Internetmedizin e.V.}
{name: Computacenter}
{name: Deutsche Telekom}
{name: Deutscher Verband f\xfcr Telekommunikation und Medien e.V.}
{name: Enquete-Kommission Internet und digitale Gesellschaft}
{name: Fujitsu Computer GmbH}
{name: I.D.I. Interessenverband Deutsches Internet. e.V.}
{name: IT-Gr\xfcnderzentrum GmbH}
{name: IT-Initiative Mecklenburg-Vorpommern e.V.}
{name: Interactive Software Federation of Europe}
{name: J Dreger IT-Operations}
{name: NextWave Telecom}
{name: Rocket Internet}
{name: SAP}
{name: Software AG}
{name: TeleTrusT- Bundesverband IT-Sicherheit e.V.}
{n

In [124]:
query = {"type": "person", "tags": "cdu"}
projection = {"_id":0, "name":1}
display(query, projection)


{name: Andreas Dombret}
{name: Andreas Krautscheid}
{name: Andreas Renner}
{name: Astrid Grotel\xfcschen}
{name: Axel Gedaschko}
{name: Cornelia Yzer}
{name: Dieter Althaus}
{name: Dietrich Birk}
{name: Eckart von Klaeden}
{name: Friedrich-Otto Ripke}
{name: Georg Brunnhuber}
{name: Hildegard M\xfcller}
{name: Joachim Koschnicke}
{name: Jost de Jager}
{name: J\xfcrgen R\xfcttgers}
{name: Leo Dautzenberg}
{name: Lothar Sp\xe4th}
{name: Matthias Wissmann}
{name: Ole von Beust}
{name: Peter Bleser}
{name: Rita Pawelski}
{name: Roland Koch}
{name: Rolf Koschorrek}
{name: Ronald Pofalla}
{name: Silke Lautenschl\xe4ger}
{name: Stefan Mappus}
{name: Tim Arnold}
{name: Ulrich Junghanns}
{name: Ursula Heinen-Esser}
{name: Volker Hoff}
{name: Walther Otremba}


In [143]:
query = {"name":"Angela Merkel"}
projection = {"_id":0, "tags":1}
display(query, projection)


{tags: [kabinette,
           kabinett-merkel-i,
           verwaltung,
           politik,
           kabinett-merkel-ii,
           kabinett-merkel-iii,
           mdb,
           bundestag,
           parteispenden,
           bundeskanzlerin]}


In [152]:
tmp = db.entities.find_one({"name":"Angela Merkel"})
p.pprint(tmp)

# vornamen: merkal, nachnamen: angela ..hemm

{_id: ObjectId('54c2a4b4fe6a42c82bbaafac'),
 aliases: [Dr. Angela Merkel, Angela Merkel],
 created: datetime.datetime(2015, 1, 23, 19, 44, 50, 772000),
 data: [{auto: True,
            created: datetime.datetime(2015, 1, 30, 13, 16, 19, 123000),
            desc: Quelle,
            format: link,
            id: 9dc681ff88ef78470f58dad0a22fde397147a9f46b416478d2fd87233fa728ab,
            key: source,
            updated: datetime.datetime(2015, 1, 30, 13, 16, 19, 123000),
            value: {remark: created by bundestag importer,
                       url: http://www.bundestag.de/bundestag/abgeordnete18/biografien/M/merkel_angela/258788}},
           {auto: True,
            created: datetime.datetime(2015, 1, 30, 13, 16, 19, 123000),
            desc: Vornamen,
            format: string,
            id: dfdee4542498db7db194d7f76a2b7912d5de8efa322e97eb49c7bee72028fe7a,
            key: names,
            updated: datetime.datetime(2015, 3, 1, 20, 26, 4, 833000),
            value: M

In [126]:
db.entities.find({"tags": None}).count()


0

In [127]:
db.entities.find({"tags": {"$exists":0}}).count()

0